In [ ]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [ ]:
extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

In [ ]:
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit)
start_date: int = int(datetime.strptime('2022-01-01', '%Y-%m-%d').timestamp())

In [ ]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [ ]:
def fetch_image(x: object) -> object:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"
		if os.path.exists(out_path):
			return out_path
		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946" or md5 in list(
				extant_data['hash']):
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		return ""

In [ ]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		print(e)
		return ""
	finally:
		img.close()

In [ ]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		print(e)
		return False

In [ ]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		print(e)
		return ""

In [ ]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		return ""
	finally:
		pass

In [ ]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		print(e)
		return ""

In [ ]:
%%time
subs = [
	# "oldladiesbakingpies",
	# 	"fatsquirrelhate",
	# 	"trippinthroughtime",
	# 	"spaceporn",
	# 	"memes",
	# 	"EarthPorn",
	# 	"CityPorn",
		"sfwpetite",
		# "SFWNextDoorGirls",
		# "SFWRedheads",
		# "RealGirls_SFW",
		# "amihot",
		# "SFWcurvy"
]

all_posts = []

for item in subs:
	posts = api.search_submissions(subreddit=item, limit=1000, filter_fn=filter_by_image_post, mem_safe=True, cache_dir='D:\\cache\\')
	[all_posts.append(post) for post in posts]

In [ ]:
%%time
df = pd.DataFrame([item for item in all_posts])

In [ ]:
%%time
display(df)

In [ ]:
%%time
initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

In [ ]:
%%time
new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

In [ ]:
%%time
new_entries_1 = new_entries.copy()

with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

In [ ]:
%%time
new_entries_2 = new_entries_1.copy()

with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

In [ ]:
%%time
blip = BlipCaption(1)

In [ ]:
%%time
new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

In [ ]:
%%time

writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

In [ ]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

In [ ]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)